In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# --- Load and Preprocess Data ---
def load_all_data(root_folder):
    all_data = []
    
    for height_folder in os.listdir(root_folder):
        height_path = os.path.join(root_folder, height_folder)
        if os.path.isdir(height_path):
            for file in os.listdir(height_path):
                if file.endswith(".mat"):
                    file_path = os.path.join(height_path, file)
                    mat_data = loadmat(file_path)
                    
                    # Extract relevant aerodynamic data
                    wind_pressure = mat_data.get('Cp_ts', np.array([])).flatten()
                    wind_angle = mat_data.get('wind_angle', np.array([])).flatten()
                    building_height_ratio = mat_data.get('building_height_ratio', np.array([])).flatten()
                    
                    if wind_pressure.size > 0 and wind_angle.size > 0 and building_height_ratio.size > 0:
                        df = pd.DataFrame({
                            'Wind_Pressure': wind_pressure,
                            'Wind_Angle': np.tile(wind_angle, len(wind_pressure) // len(wind_angle)),
                            'Building_Height_Ratio': np.tile(building_height_ratio, len(wind_pressure) // len(building_height_ratio))
                        })
                        all_data.append(df)
    
    return pd.concat(all_data, ignore_index=True) if all_data else None

# Load all data
data_folder = "../data/Low-rise with eaves/roof type a/height 1;4"  # Root folder containing height ratio subfolders
df = load_all_data(data_folder)

if df is None:
    raise ValueError("No valid data found in the provided folder structure.")

features = df[['Wind_Angle', 'Building_Height_Ratio']]
targets = df['Wind_Pressure']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define PINN Model
class PINN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PINN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize Model
input_dim = X_train.shape[1]
model = PINN(input_dim, 128, 1).to(device)

# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Convert to Tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Training Loop
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Evaluation
model.eval()
y_pred_tensor = model(X_test_tensor).cpu().detach().numpy()
y_pred = y_pred_tensor.flatten()

ae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {ae:.4f}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

# Visualization
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r', lw=2)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs. Predicted Wind Pressure")
plt.show()

sns.histplot(y_test - y_pred, bins=30, kde=True)
plt.xlabel("Prediction Error")
plt.title("Prediction Error Distribution")
plt.show()

# Function to Predict Mean Wind Pressure Coefficient
def predict_wind_pressure(wind_angle, building_height_ratio):
    model.eval()
    input_data = np.array([[wind_angle, building_height_ratio]])
    input_scaled = scaler.transform(input_data)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)
    prediction = model(input_tensor).cpu().detach().numpy().flatten()[0]
    return prediction

# Example usage:
wind_angle_test = 45.0  # Example wind angle
building_height_ratio_test = 0.6  # Example height ratio
predicted_pressure = predict_wind_pressure(wind_angle_test, building_height_ratio_test)
print(f"Predicted Mean Wind Pressure Coefficient: {predicted_pressure:.4f}")


Using device: cuda


ValueError: No valid data found in the provided folder structure.